### Heroes Of Pymoli Data Analysis
* Of the 1163 active players, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.8%) with secondary groups falling between 15-19 (18.60%) and 25-29 (13.4%).  
-----

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchases_data_df = pd.read_csv(file_to_load)

## Player Count

* Display the total number of players


In [2]:
unique_users_df = purchases_data_df['SN'].unique()
unique_users_count = len(unique_users_df)
print(f'Players count: {unique_users_count}')

Players count: 576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [3]:
unique_items_list = purchases_data_df['Item ID'].unique()
unique_items_count = len(unique_items_list)

average_price = purchases_data_df['Price'].mean()

number_of_purchases = len(purchases_data_df.index)

total_revenue = number_of_purchases * average_price

data_frame_structure = {
    'Number of Unique Items' : [unique_items_count],
    'Average Price' : [average_price],
    'Number of Purchases' : [number_of_purchases],
    'Total Revenue' : [total_revenue]
}

purchasing_analysis_df = pd.DataFrame(data=data_frame_structure)

purchasing_analysis_df = purchasing_analysis_df.style.format(
   {
       'Average Price': "${:.2f}",
       'Total Revenue': '${:.2f}'
   }
)

purchasing_analysis_df


,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,$2379.77


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [4]:
genders_list = purchases_data_df['Gender'].unique()
genders_list.sort()


male_players_df = purchases_data_df.loc[purchases_data_df['Gender'] == 'Male']
unique_male_players_list = male_players_df['SN'].unique()
unique_male_players_count = len(unique_male_players_list)

female_players_df = purchases_data_df.loc[purchases_data_df['Gender'] == 'Female']
unique_female_players_list = female_players_df['SN'].unique()
unique_female_players_count = len(unique_female_players_list)

other_players_condition = (purchases_data_df['Gender'] != 'Male') & (purchases_data_df['Gender'] != 'Female')
other_players_df = purchases_data_df.loc[other_players_condition]
unique_other_players_list = other_players_df['SN'].unique()
unique_other_players_count = len(unique_other_players_list)

data_frame_structure = {
    'Gender' : genders_list,
    'Total Count' : [unique_female_players_count, unique_male_players_count, unique_other_players_count],
    'Percentage of Players' : [
        (unique_female_players_count/unique_users_count*100),
        (unique_male_players_count/unique_users_count*100),
        (unique_other_players_count/unique_users_count*100)
    ]
}

gender_demographics_with_df = pd.DataFrame(data=data_frame_structure)
gender_demographics_with_df = gender_demographics_with_df.set_index('Gender')
gender_demographics_with_format = gender_demographics_with_df.copy()
gender_demographics_with_format = gender_demographics_with_format.style.format({'Percentage of Players': "{:.2f}%"})

gender_demographics_with_format

,Total Count,Percentage of Players
Gender,,
Female,81,14.06%
Male,484,84.03%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [13]:
purchasing_analysis_by_gender_df = purchases_data_df.groupby(['Gender']).agg(
    {
        'Purchase ID' : 'count',
        'Price' : 'mean',
    }
)

purchasing_analysis_by_gender_df = purchasing_analysis_by_gender_df.rename(
    columns={
        'Purchase ID': 'Purchase Count',
        'Price' : 'Average Purchase Price'
    }
)

purchasing_analysis_by_gender_df['Total Purchase Value'] = (
    purchasing_analysis_by_gender_df['Purchase Count'] * purchasing_analysis_by_gender_df['Average Purchase Price']
)
purchasing_analysis_by_gender_df['Avg Total Purchase per Person'] = (
    purchasing_analysis_by_gender_df['Total Purchase Value'] / gender_demographics_with_df['Total Count']
)

purchasing_analysis_by_gender_df = purchasing_analysis_by_gender_df.style.format(
   {
       'Average Purchase Price': "${:.2f}",
       'Total Purchase Value': "${:.2f}",
       'Avg Total Purchase per Person': "${:.2f}"
   }
)

purchasing_analysis_by_gender_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [14]:
age_bins = [0, 9.9, 14, 19, 24, 29, 34, 39, 100]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

purchases_data_df['Age Ranges'] = pd.cut(purchases_data_df['Age'], age_bins, labels=group_names)

removed_dupplicate_users = purchases_data_df.copy()
removed_dupplicate_users = removed_dupplicate_users.sort_values('SN')
removed_dupplicate_users = removed_dupplicate_users.drop_duplicates(subset = 'SN', keep = 'first')

removed_dupplicate_users = removed_dupplicate_users[['Age Ranges','Age']]
removed_dupplicate_users = removed_dupplicate_users.set_index('Age Ranges')

age_demographics_df = removed_dupplicate_users.groupby(['Age Ranges']).count()
age_demographics_df = age_demographics_df.rename(columns={'Age': 'Total Count'})
age_demographics_df['Percentage of Players'] = (
    age_demographics_df['Total Count'] / age_demographics_df['Total Count'].sum()*100
)

age_demographics_df_with_format = age_demographics_df.copy()
age_demographics_df_with_format = age_demographics_df_with_format.style.format({'Percentage of Players': "{:.2f}%"})

age_demographics_df_with_format


,Total Count,Percentage of Players
Age Ranges,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [15]:
purchasing_analysis_by_age = purchases_data_df.groupby(['Age Ranges']).agg(
    {
        'Purchase ID' : 'count',
        'Price' : 'mean',
    }
)

purchasing_analysis_by_age = purchasing_analysis_by_age.rename(columns={"Purchase ID": "Purchase Count", "Price" : "Average Purchase Price"})

purchasing_analysis_by_age['Total Purchase Value'] = purchasing_analysis_by_age['Purchase Count'] * purchasing_analysis_by_age['Average Purchase Price']
purchasing_analysis_by_age['Avg Total Purchase per Person'] = purchasing_analysis_by_age['Total Purchase Value'] / age_demographics_df['Total Count']

purchasing_analysis_by_age = purchasing_analysis_by_age.style.format(
   {
       'Average Purchase Price': "${:.2f}",
       'Total Purchase Value': "${:.2f}",
       'Avg Total Purchase per Person': "${:.2f}"
   }
)

purchasing_analysis_by_age

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Ranges,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [16]:
top_spenders_df = purchases_data_df.groupby(['SN']).agg(
    {
        'Purchase ID' : 'count',
        'Price' : 'mean'
    }
)

top_spenders_df = top_spenders_df.rename(
    columns = {
        'Purchase ID' : 'Purchase Count',
        'Price' : 'Average Purchase Price'
    }
)

top_spenders_df['Total Purchase Value'] = (
    top_spenders_df['Purchase Count'] * top_spenders_df['Average Purchase Price']
)
top_spenders_df = top_spenders_df.sort_values(by=['Total Purchase Value'], ascending=False)

top_spenders_df = top_spenders_df.iloc[0:5]

top_spenders_df = top_spenders_df.style.format(
    {
        'Average Purchase Price': "${:.2f}",
        'Total Purchase Value': "${:.2f}"
    }
)

top_spenders_df


,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [17]:
items_only_df = purchases_data_df[['Item ID','Item Name','Price']]
items_only_df = items_only_df.groupby(['Item ID', 'Item Name']).agg({'Item ID': 'count', 'Price' : 'mean'})
items_only_df = items_only_df.rename(columns={'Item ID': 'Purchase Count', 'Price': 'Item Price'})
items_only_df['Total Purchase Value'] = items_only_df['Purchase Count'] * items_only_df['Item Price']

most_popular_items_df = items_only_df.sort_values(by=['Purchase Count'], ascending=False)

most_popular_items_df = most_popular_items_df.iloc[0:5]
most_popular_items_df = most_popular_items_df.style.format(
    {
        'Item Price': "${:.2f}",
        'Total Purchase Value': "${:.2f}"
    }
)

most_popular_items_df


,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [18]:
most_profitable_items_df = items_only_df.sort_values(by=['Total Purchase Value'], ascending=False)

most_profitable_items_df = most_profitable_items_df.iloc[0:5]
most_profitable_items_df = most_profitable_items_df.style.format(
    {
        'Item Price': "${:.2f}",
        'Total Purchase Value': "${:.2f}"
    }
)

most_profitable_items_df

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
